In [6]:
import torch 
import torch.nn as nn
import torch.optim as optim  



In [4]:
print(torch.__version__)

2.6.0+cpu


In [ ]:
def make_batch():
    input_batch=[]
    target_batch=[]
    for sen in sentences:
        word=sen.split()
        input=[word_dict[n] for n in word[::-1]]
        target=word_dict[word[-1]]
        input_batch.append(input)
        target_batch.append(target)

    return input_batch,target_batch


class NNLM(nn.Module):
    def __init__(self):
        super(NNLM,self).__init__()
        self.C=nn.Embedding(n_class,m)
        self.H=nn.Linear(n_step*m,n_hidden,bias=False)
        self.d=nn.Parameter(torch.ones(n_hidden))
        self.U=nn.Linear(n_hidden,n_class,bias=False)
        self.W=nn.Linear(n_step*m,n_class,bias=False)
        self.b=nn.Parameter(
        